In [0]:
spark

In [0]:
dbutils.library.restartPython()
# %pip install libpysal
%pip install splot
%pip install esda

## Gráficos
Realizamos los siguientes graficos como aporte a la solución planteada en la POC de UnalWater
1. Grafico de dispersión de los puntos
2. Mapa de cloropletas por barrio
3. Mapa de densidad de Kernel
4. Histograma de cantidad de productos vendidos
5. Boxplot de cantidad de productos vendidos por barrios
5. Histograma de productos vendidos por horas

In [0]:
# Importamos las librerías necesarias
from shapely.validation import make_valid
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from libpysal import weights
import esda
from splot.esda import lisa_cluster

In [0]:
# Paso 0: Importamos y editamos los nombres de los barrios

path_parquet_neigh = "/Workspace/Users/danielale22rojas@gmail.com//medellin-bigdata-poc/data/raw/medellin_neighborhoods.parquet"
gdf_barrios = gpd.read_parquet(path_parquet_neigh) 

gdf_barrios["geometry"] = gdf_barrios["geometry"].apply(make_valid)

# Primero corregimos el nombre
gdf_barrios.loc[gdf_barrios["NOMBRE"].isna(), "NOMBRE"] = "ARANJUEZ"

# Luego disolvemos por el nombre
gdf_barrios = gdf_barrios.dissolve(by="NOMBRE", as_index=False)

gdf_barrios["NOMBRE"] = gdf_barrios["NOMBRE"].str.replace("CORREGIMIENTO DE ", "", regex=True)
gdf_barrios["NOMBRE"]

In [0]:
# Paso 1: Creamos una función para importar la capa oro como un geodataframe

def cargar_silver_como_gdf(nombre_tabla="poctesting.gold_events", crs="EPSG:4326"):
    """
    Carga la tabla Silver desde Spark y la convierte en un GeoDataFrame.
    """
    # Leer la tabla desde Spark
    df_gold = spark.table(nombre_tabla)
    
    # Pasar a Pandas
    pdf = df_gold.toPandas()

    # Crear el "geometry" a partir de lon y lat
    pdf["geometry"] = pdf.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)
    
    # Construir GeoDataFrame
    gdf = gpd.GeoDataFrame(pdf, geometry="geometry", crs=crs)
    
    return gdf

In [0]:
# Importando los datos desde oro
# bd es la base de datos gold importada como un GeodataFrame de pandas
bd = cargar_silver_como_gdf("poctesting.gold_events", "EPSG:4326")

In [0]:
# Particionar las bases de datos, por barrios y por empleados


### 1. Gráfico de dispersión de los puntos

In [0]:
# Crear el jointplot
plot = sbn.jointplot(
    x='longitude', 
    y='latitude', 
    data=bd, 
    s=5, 
    height=8
)

# Obtener el eje principal del jointplot
ax = plot.ax_joint

# Dibujar los límites de barrios encima
gdf_barrios.boundary.plot(ax=ax, color="black", linewidth=0.5)

# Personalizar títulos y etiquetas
plot.fig.suptitle("Distribución de puntos de venta con límites de barrios", y=1.02, fontsize=16)
plot.set_axis_labels("Longitud", "Latitud")

plt.show()

### 2. Mapa de cloropletas por barrio

In [0]:
# crear dataframe unido
df_gold = spark.table("poctesting.gold_events")
pdf_gold = df_gold.select("district", "avg_by_neighborhood", "total_by_neighborhood").distinct().toPandas()

# Unir los dos dataframes
gdf_merged = gdf_barrios.merge(pdf_gold, left_on="NOMBRE", right_on="district", how="left")

In [0]:
# Importar datos gold

# Grafico coroplético
f, ax = plt.subplots(1, figsize=(12,7))
gdf_merged.plot(
    ax=ax,
    column="avg_by_neighborhood",  # usamos el total ya calculado
    legend=True,
    scheme="Quantiles",
    legend_kwds={"fmt": "{:.0f}"},
    cmap="Blues",
    edgecolor="black",
    linewidth=0.5
)

# Dibujar límites de barrios
gdf_barrios.boundary.plot(ax=ax, color="black", linewidth=0.5, alpha=0.5)

# Añadir nombres de barrios
for idx, row in gdf_barrios.iterrows():
    centroid = row.geometry.centroid
    ax.text(
        centroid.x, centroid.y, 
        str(row["NOMBRE"]), 
        fontsize=6, color="black", ha="center"
    )

ax.set_axis_off()
ax.set_title("Total de productos vendidos por barrio", fontsize=14)
plt.axis("equal")
plt.show()

### 3. Mapa de densidad de Kernel

In [0]:
from matplotlib.cm import ScalarMappable

# Crear figura
f, ax = plt.subplots(figsize=(10, 12))

# KDE con seaborn
sns_plot = sbn.kdeplot(
    x=bd["longitude"], 
    y=bd["latitude"], 
    fill=True, 
    cmap="viridis_r", 
    levels=40, 
    alpha=0.7, 
    ax=ax
)

# Dibujar límites de barrios
gdf_barrios.boundary.plot(ax=ax, color="black", linewidth=0.5, alpha=0.5)

# Añadir nombres de barrios
for idx, row in gdf_barrios.iterrows():
    centroid = row.geometry.centroid
    ax.text(
        centroid.x, centroid.y, 
        str(row["NOMBRE"]), 
        fontsize=7, color="black", ha="center"
    )

# Ajustar límites al bounding box de Medellín
bounds = gdf_barrios.total_bounds
ax.set_xlim(bounds[0], bounds[2])
ax.set_ylim(bounds[1], bounds[3])

# Crear colorbar manual (gradiente)
sm = ScalarMappable(cmap="viridis_r")
sm.set_array([])  # necesario para inicializar
cbar = f.colorbar(sm, ax=ax, orientation="vertical", fraction=0.03, pad=0.04)
cbar.set_label("Densidad estimada de eventos", fontsize=12)

# Estilo final
ax.set_title("Mapa de densidad de eventos de ventas en Medellín", fontsize=16, pad=20)
ax.set_axis_off()
plt.tight_layout()
plt.show()

In [0]:
from pointpats import QStatistic

# Mapa de distribución de puntos en cuadrantes
coordinates = bd[["longitude", "latitude"]].values
qstat = QStatistic(coordinates, nx = 2, ny = 2)
qstat.plot()

### 4. Histograma de cantidad de productos vendidos

In [0]:
# Histograma
plt.figure(figsize=(10,6))
plt.hist(bd["quantity_products"], bins=30, color="skyblue", edgecolor="black")
plt.title("Histograma de productos vendidos - Poisson No homogenea", fontsize=16)
plt.xlabel("Cantidad de productos")
plt.ylabel("Frecuencia")
plt.grid(alpha=0.3)
plt.show()

In [0]:
# Boxplot
# Calcular medianas de productos vendidos por barrio
orden = bd.groupby("district")["quantity_products"].median().sort_values().index

# Crear el boxplot con orden
plt.figure(figsize=(14,7))
sbn.boxplot(data=bd, x="district", y="quantity_products", order=orden)
plt.title("Boxplot de productos vendidos por barrio (ordenado por mediana)", fontsize=16)
plt.xlabel("Barrio")
plt.ylabel("Cantidad de productos")
plt.xticks(rotation=90)
plt.show()

### 5. Dependencia Espacial y Clustering

In [0]:
# Creamos la matriz de pesos espacial tipo queen
w_queen = weights.Queen.from_dataframe(gdf_merged)

In [0]:
# Grafico de moran con matriz de pesos queen

from splot.esda import plot_moran

plot_moran(esda.Moran(gdf_merged['total_by_neighborhood'], w_queen));

In [0]:
from splot.esda import lisa_cluster

# Ahora hacemos el grafico de lisa
lisa = esda.Moran_Local(gdf_merged['total_by_neighborhood'], w_queen)
lisa_cluster(lisa, gdf_merged);

In [0]:
from splot.esda import plot_local_autocorrelation
plot_local_autocorrelation(lisa, gdf_merged, 'total_by_neighborhood');

### Culstering

In [0]:
# CLUSTERING CON DBSCAN
from sklearn.cluster import DBSCAN
import numpy as np

# Extraer coordenadas
coords = np.array(list(zip(bd.geometry.x, bd.geometry.y)))

# DBSCAN con distancia en coordenadas
db = DBSCAN(eps=0.01, min_samples=5).fit(coords)  
bd["cluster"] = db.labels_

# Revisar resultados
print(bd["cluster"].value_counts())

# Graficar
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,8))
bd.plot(ax=ax, column="cluster", categorical=True, legend=True, markersize=10, cmap="tab20")
ax.set_title("Clustering de eventos (DBSCAN)", fontsize=14)
plt.show()

In [0]:
# CLUSTERING CON K-MEANS
from sklearn.cluster import KMeans

coords = np.array(list(zip(bd.geometry.x, bd.geometry.y)))
kmeans = KMeans(n_clusters=5, random_state=42).fit(coords)
bd["cluster"] = kmeans.labels_

# Graficar
fig, ax = plt.subplots(figsize=(10,8))
bd.plot(ax=ax, column="cluster", categorical=True, legend=True, markersize=10, cmap="tab20")
ax.set_title("Clustering de eventos (K-Means)", fontsize=14)
plt.show()

In [0]:
# CLUSTERING CON DBSCAN CON BARRIOS
from sklearn.cluster import DBSCAN
import numpy as np

# Extraer coordenadas
coords = np.array(list(zip(gdf_merged.geometry.x, gdf_merged.geometry.y)))

# DBSCAN con distancia en coordenadas
db = DBSCAN(eps=0.01, min_samples=5).fit(coords)  
gdf_merged["cluster"] = db.labels_

# Revisar resultados
print(gdf_merged["cluster"].value_counts())

# Graficar
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,8))
gdf_merged.plot(ax=ax, column="cluster", categorical=True, legend=True, markersize=10, cmap="tab20")
ax.set_title("Clustering de eventos (DBSCAN)", fontsize=14)
plt.show()